In [1]:
import numpy as np
import sklearn
import sklearn.pipeline as skpipe
# learners
import celer as cel
from lightgbm import LGBMClassifier, LGBMRegressor

# this module
from aipyw import AIPyW
from aipyw.dgp import dgp_binary, dgp_discrete, hainmueller

np.random.seed(42)

# Basic Demo


## Discrete Treatments

In [4]:
true_effects = np.array([0.0, 0.4, 0.5, 0.55])
Y, D, X = dgp_discrete(
    n=100_000,
    p=4,
    treat_effects=true_effects,
)
Y.shape, D.shape, X.shape

((100000,), (100000,), (100000, 10))

In [3]:
(Y[D == 1,].mean() - Y[D == 0,].mean(),
  Y[D == 2,].mean() - Y[D == 0,].mean(),
  Y[D == 3,].mean() - Y[D == 0,].mean()
)

(-2.459176579755844, 0.9312130397763212, 2.8166707333716685)

Naive estimates badly biased.

In [6]:
%%time
doubledouble3 = AIPyW(riesz_method="ipw")
doubledouble3.fit(X, D, Y)
doubledouble3.summary()

CPU times: user 1.98 s, sys: 2.74 s, total: 4.72 s
Wall time: 334 ms


{'1 vs 0': {'effect': 0.3994630943262704, 'se': 1.3622589430481528e-05},
 '2 vs 0': {'effect': 0.5003776701426296, 'se': 1.4060143957220782e-05},
 '3 vs 0': {'effect': 0.5491231362790109, 'se': 1.3103465903159675e-05},
 '2 vs 1': {'effect': 0.10091457581635918, 'se': 1.541930994660243e-05},
 '3 vs 1': {'effect': 0.14966004195274035, 'se': 1.5486070939316562e-05},
 '3 vs 2': {'effect': 0.048745466136381144, 'se': 1.9273141207355643e-05}}

In [7]:
%%time
doubledouble3 = AIPyW(riesz_method="linear")
doubledouble3.fit(X, D, Y)
doubledouble3.summary()

CPU times: user 32.6 s, sys: 11.9 s, total: 44.5 s
Wall time: 3.13 s


{'1 vs 0': {'effect': 0.3516921176737457, 'se': 0.0003074449246188787},
 '2 vs 0': {'effect': 0.442799370180441, 'se': 0.00032876940551221443},
 '3 vs 0': {'effect': 0.48361785088438103, 'se': 0.0003367267795149449},
 '2 vs 1': {'effect': 0.09110725250669531, 'se': 0.0002464474798193398},
 '3 vs 1': {'effect': 0.13192573321063542, 'se': 0.00026858923244345995},
 '3 vs 2': {'effect': 0.04081848070394008, 'se': 0.0002802135791847662}}

In [8]:
%%time
doubledouble3 = AIPyW(riesz_method="balancing", bal_obj="quadratic")
doubledouble3.fit(X, D, Y)
doubledouble3.summary()

CPU times: user 1.38 s, sys: 1.69 s, total: 3.07 s
Wall time: 230 ms


{'1 vs 0': {'effect': 0.39851183324945333, 'se': 1.4484548390155508e-05},
 '2 vs 0': {'effect': 0.4987931530559083, 'se': 1.4243642678264582e-05},
 '3 vs 0': {'effect': 0.5473111937733468, 'se': 1.3152371384222194e-05},
 '2 vs 1': {'effect': 0.10028131980645509, 'se': 1.628660556388105e-05},
 '3 vs 1': {'effect': 0.14879936052389353, 'se': 1.6086796143714927e-05},
 '3 vs 2': {'effect': 0.04851804071743844, 'se': 1.9199184339119927e-05}}

## Hainmueller (2012) Simulation study

Binary treatment, continuous outcome, 2 groups. We parametrize degree of overlap, functional form of outcome and treatment models. True effect is zero, so RMSE is easy to calculate.

In [10]:
def one_rep(
    n_samples, overlap_design, pscore_design, outcome_design, riesz_method, **kwargs
):
    # generate data
    y, d, X = hainmueller(
        n_samples=n_samples,
        overlap_design=overlap_design,
        pscore_design=pscore_design,
        outcome_design=outcome_design,
    )
    m1, m2 = LGBMRegressor(verbose=-1, n_jobs=1), LGBMClassifier(verbose=-1, n_jobs=1)
    # model instantiation
    aipw = AIPyW(
        propensity_model=m2, outcome_model=m1, riesz_method=riesz_method, **kwargs
    )
    aipw.fit(X, d, y, n_rff=100)
    return aipw.summary()["1 vs 0"]["effect"]

Favorable case: good overlap, linear pscore and outcome

In [11]:
%%time
one_rep(10_000, 2, 1, 1, "ipw")

CPU times: user 374 ms, sys: 3.85 ms, total: 378 ms
Wall time: 391 ms


0.0836386558119037

In [12]:
%%time
one_rep(10_000, 2, 1, 1, "linear")

CPU times: user 3.87 s, sys: 1.58 s, total: 5.46 s
Wall time: 612 ms


0.015526093837424577

In [13]:
%%time
one_rep(10_000, 2, 1, 1, "kernel")

CPU times: user 10.7 s, sys: 2.63 s, total: 13.4 s
Wall time: 1.19 s


0.026719792404255066

In [14]:
%%time
one_rep(10_000, 2, 1, 1, "automatic")

CPU times: user 277 ms, sys: 22.4 ms, total: 300 ms
Wall time: 270 ms


0.009155275845615768

In [15]:
%%time
one_rep(10_000, 2, 1, 1, "balancing")

CPU times: user 302 ms, sys: 43.2 ms, total: 345 ms
Wall time: 278 ms


0.025431150590310692

In [16]:
%%time
one_rep(10_000, 2, 1, 1, "balancing", bal_obj="entropy")

CPU times: user 565 ms, sys: 500 ms, total: 1.06 s
Wall time: 332 ms


0.03250301580285296

### Hard case: poor overlap, non-linear pscore and outcome

In [17]:
%%time
one_rep(10_000, 1, 3, 3, "ipw")

CPU times: user 412 ms, sys: 358 µs, total: 412 ms
Wall time: 412 ms


69.44373946330548

In [18]:
%%time
one_rep(10_000, 1, 3, 3, "linear")

CPU times: user 4.29 s, sys: 1.78 s, total: 6.07 s
Wall time: 650 ms


0.09306011349615892

In [19]:
%%time
one_rep(10_000, 1, 3, 3, "kernel")

CPU times: user 15.3 s, sys: 2.87 s, total: 18.2 s
Wall time: 1.47 s


-1.7437454155475818

In [20]:
%%time
one_rep(10_000, 2, 1, 1, "automatic")

CPU times: user 285 ms, sys: 11.8 ms, total: 297 ms
Wall time: 279 ms


0.05647212246812064

In [21]:
%%time
one_rep(10_000, 1, 3, 3, "balancing")

CPU times: user 283 ms, sys: 21.9 ms, total: 305 ms
Wall time: 263 ms


0.3810860547495119

In [22]:
%%time
one_rep(10_000, 1, 3, 3, "balancing", bal_obj="entropy")

CPU times: user 392 ms, sys: 179 ms, total: 571 ms
Wall time: 273 ms


-1.7983822878296425

### all together

In [23]:
from joblib import Parallel, delayed

def compute_ate_rmse_parallel(
    n_samples,
    overlap_design,
    pscore_design,
    outcome_design,
    riesz_method,
    n_replications=100,
    n_jobs=-1,
):
    ate_estimates = Parallel(n_jobs=n_jobs)(
        delayed(one_rep)(
            n_samples, overlap_design, pscore_design, outcome_design, riesz_method
        )
        for _ in range(n_replications)
    )
    # Compute RMSE
    true_ate = 0
    rmse = np.sqrt(np.mean((np.array(ate_estimates) - true_ate) ** 2))
    return rmse

In [24]:
%%time
from itertools import product
params = np.arange(1, 4)
param_list = list(product(params, params, params, ['ipw', 'linear', 'kernel', 'automatic', 'balancing']))
res_dict = {}
for param in param_list:
  key = "_".join([str(x) for x in param])
  res_dict[key] = compute_ate_rmse_parallel(10_000, *param)

CPU times: user 15.4 s, sys: 3.42 s, total: 18.8 s
Wall time: 19min 18s


In [25]:
import pandas as pd

res_df = pd.DataFrame(
    [
        list(
            product(
                ["poor", "good", "medium"],
                ["linear", "quad", "trig"],
                ["linear", "quad", "nl"],
            )
        ),
        [v for k, v in res_dict.items() if k.endswith("ipw")],
        [v for k, v in res_dict.items() if k.endswith("linear")],
        [v for k, v in res_dict.items() if k.endswith("kernel")],
        [v for k, v in res_dict.items() if k.endswith("automatic")],
        [v for k, v in res_dict.items() if k.endswith("balancing")],
    ],
).T.infer_objects()
res_df.columns = ["design", "ipw", "linear", "kernel", "automatic", "constrained"]
# unpack design column
res_df["overlap_design"] = res_df["design"].apply(lambda x: x[0])
res_df["pscore_design"] = res_df["design"].apply(lambda x: x[1])
res_df["outcome_design"] = res_df["design"].apply(lambda x: x[2])
res_df.drop(columns=["design"], inplace=True)

res_df

,ipw,linear,kernel,automatic,constrained,overlap_design,pscore_design,outcome_design
0,0.067663,0.069738,0.038778,0.062457,0.065027,poor,linear,linear
1,0.061450,0.065838,0.034477,0.067643,0.064520,poor,linear,quad
2,8.026715,11.399964,4.230426,23.012962,34.992916,poor,linear,nl
3,0.060049,0.061884,0.039053,0.058086,0.051982,poor,quad,linear
4,0.036830,0.035757,0.022515,0.040318,0.033273,poor,quad,quad
5,18.876822,5.905107,183.514660,711.297691,13.271389,poor,quad,nl
6,0.023669,0.012367,0.011074,0.028292,0.024507,poor,trig,linear
7,0.023306,0.011653,0.012639,0.021560,0.025335,poor,trig,quad
8,22.429755,8.371317,23.645728,31.252971,238.541241,poor,trig,nl
9,0.034867,0.029071,0.017621,0.032041,0.031465,good,linear,linear
